- (o) target transformation
- (o) group scaling
- interaction term
- feature selection
- outlier detection
- (o) memory reduce
- (o) hour기준 시간대정보: "아침","오후","저녁","밤"
- (o) 기온,강수량,풍속,습도로 파생변수 생성
- (o) 주기성정보: sin_hour, cos_hour
- (x) 전일차이 & rolling mean/std -> test까지 고려해서 넣어야하므로 복잡함
- (o) stacking
- kfold -> timeseries kfold
- (x) lag variable -> 속도만느려지는거같음
- (x) target lag variable -> 성능 매우 안좋아짐
- weiens -> cv추가
- xgb,lgb는 성능이 안좋음 -> 제거
- stacking에서 5개평균으로 넣어보기

# Setting

In [1]:
import sys
sys.path.append('/Volumes/KHJ/Github/hyuckjinkim/lib-python')

In [2]:
from base import gc_collect_all
from data_prepare import (
    get_holiday, reduce_mem_usage, delete_unique_columns,
    TypeController, CategoricalQuantileCalculator,
    GroupScaler, OneHotEncoder, InteractionTerm, TargetTransform,
)

In [3]:
gc_collect_all()

In [5]:
from tqdm import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

import datetime

pd.set_option('mode.chained_assignment',  None)

In [6]:
def global_assignment(dictionary):
    for k,v in dictionary.items():
        exec("globals()['{}']=dictionary['{}']".format(k,k))

In [7]:
def smape(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [8]:
import numpy as np

class TargetTransfom:
    def __init__(self,transform='identity'):
        assert transform in ['identity','log','sqrt'], \
            print("transform must be one of ['identity','log','sqrt']")
        self.transform = transform
        
    def fit(self,target,verbose=True):
        x = np.array(target)
        if min(x)>0:
            self.offset = 0
        elif min(x)==0:
            self.offset = 1e-3
        else:
            self.offset = -min(x)+1e-3
            if verbose:
                print('minimum = {:.3f}'.format(min(x)))
        #self.offset)return np.log(x+
    
    def transform(self,target):
        x = np.array(target)
        return np.log(x+self.offset)
    
    def fit_transform(self,target,verbose):
        self.fit(self,target,verbose)
        return self.transform(target)

<br></br>

# Configuration

In [9]:
class CFG:
    TARGET = '전력소비량(kWh)'
    SEGMENT = '건물유형'
    
    # feature engineering
    SUBSET_DEPTH = 3
    TARGET_TRANSFORMATION = 'log'#'identity'
    
    # kfold
    SEED = 0
    N_SPLITS = 5

<br></br>

# Data Load

In [10]:
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')
info_df  = pd.read_csv('./data/building_info.csv')

<br></br>

# EDA

## numeric 1D

(1) 건물번호/건물번호별로 날짜에따른 컬럼 & 타켓 lineplot

In [11]:
def multi_lineplot(data,x,target,columns):
    columns = [col for col in columns if col not in target]
    
    nrow = int(np.ceil(np.sqrt(len(columns))))
    ncol = int(np.ceil(len(columns)/nrow))

    fig = plt.figure(figsize=(20,15))

    for i,col in tqdm(enumerate(columns),total=len(columns)):
        fig.add_subplot(nrow,ncol,i+1)
        ax1 = plt.twinx()
        ax2 = ax1.twinx()
        sns.lineplot(x=date,y=data[col],ax=ax1,color='gray')
        sns.lineplot(x=date,y=data[target],ax=ax2,color='red',alpha=0.5)
        plt.xticks(rotation=45)
        plt.grid()
        plt.title(col,fontsize=20)

    plt.tight_layout()
    plt.show()
    
def make_datetime(yyyy,mm,dd,hh):
    date = str(int(yyyy)) + str(int(mm)).zfill(2) + str(int(dd)).zfill(2) + str(int(hh)).zfill(2)
    date = datetime.datetime.strptime(date,'%Y%m%d%H')
    return date

In [37]:
# group = '건물번호'
# g_list = train_df3[group].unique()

# i=0
# for g in g_list:
#     i+=1
#     print('\n> [{}/{}] {}: {}'.format(str(i).zfill(len(str(len(g_list)))),len(g_list),group,g))
    
#     d = train_df[train_df[group]==g]
#     d = d.astype({'hour':int})
#     cols = d.columns[d.dtypes!='object'].tolist()
#     cols = [c for c in cols if c not in ['year','month','day','hour','건물번호']]
    
#     date = [make_datetime(yyyy,mm,dd,hh) for yyyy,mm,dd,hh in d[['year','month','day','hour']].values]
#     multi_lineplot(d,date,CFG.TARGET,cols)

(2) 건물번호/건물번호별로 시간에따른 컬럼 & 타켓 boxplot

In [40]:
# group = ['건물번호','건물유형'][1]
# g_list = train_df3[group].unique()

# i=0
# for g in g_list:
#     i+=1
#     print('\n> [{}/{}] {}: {}'.format(str(i).zfill(len(str(len(g_list)))),len(g_list),group,g))
    
#     d = train_df[train_df[group]==g]
#     d = d.astype({'hour':int})
#     columns = d.columns[d.dtypes!='object'].tolist()
#     columns = [c for c in columns if c not in ['year','month','day','hour',CFG.TARGET]]

#     nrow = int(np.ceil(np.sqrt(len(columns))))
#     ncol = int(np.ceil(len(columns)/nrow))

#     fig = plt.figure(figsize=(25,20))
#     for i,col in enumerate(columns):
#         fig.add_subplot(nrow,ncol,i+1)
#         ax1 = plt.twinx()
#         ax2 = ax1.twinx()
#         sns.boxplot(x=d['hour'],y=d[CFG.TARGET],ax=ax1)
#         sns.lineplot(x=d['hour'],y=d[col],ax=ax2)
#         plt.title(col,fontsize=20)

#     plt.tight_layout()
#     plt.show()

<br></br>

# Data Preprocess

## Derived Features

In [14]:
def derived_features(data,info):
    d = data.copy()

    # (1) join the building_info
    d = pd.merge(d,info,how='left',on='건물번호')

    if len(d)!=len(data):
        raise ValueError('duplicated after join the building_info')

    # (2) delete the unuse columns
    d.drop('num_date_time',axis=1,inplace=True)

    # (3) 날짜관련 파생변수
    d['일시'] = d['일시'].apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d %H'))
    
    # (3-1) 날짜관련 변수
    d['year']    = d['일시'].dt.year
    d['month']   = d['일시'].dt.month
    d['day']     = d['일시'].dt.day
    d['hour']    = d['일시'].dt.hour
    d['weekday'] = d['일시'].dt.weekday
    d['weekend'] = np.where((d['weekday']==5)|(d['weekday']==6),1,0)
    
    # (3-2) 한국 공휴일
    kr_holidays = get_holiday(d['year'].unique())
    d['is_holiday'] = d['일시'].apply(lambda x: np.where(datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d') in kr_holidays,1,0))
    
    # (3-3) 날짜변수 삭제
    d.drop('일시',axis=1,inplace=True)
    
    # (4) 주기성 정보
    d['sin_hour'] = np.sin(2 * np.pi * (d['hour']+1) / 24)
    d['cos_hour'] = np.cos(2 * np.pi * (d['hour']+1) / 24)
    
    # (5) 시간대 정보
    d['time_of_day'] = ['morning'   if  5<=hour<12 else
                        'afternoon' if 12<=hour<17 else
                        'evening'   if 17<=hour<21 else
                        'night' for hour in d['hour']]
    
    # (6) 강수여부
    d['강수량(mm)'].fillna(0,inplace=True)
    d['is_rain'] = np.where(d['강수량(mm)']==0,1,0)
    
    # # 강수량 그룹 : https://namu.wiki/w/%EA%B0%95%EC%9A%B0%EB%9F%89#s-2
    # # -> 그룹이 너무 많아서 제거
    # d['rain_group'] = ['비오지않음' if x<=0 else
    #                    '흩날리는 비' if x<=1 else
    #                    '가늘게 내리는 비' if x<=3 else
    #                    '부슬부슬 내리는 비' if x<=5 else
    #                    '추적추적 내리는 비' if x<=10 else
    #                    '굵은 비' if x<=15 else
    #                    '요란한 비' if x<=20 else
    #                    '장대비' if x<=30 else
    #                    '쏟아붇는 비' if x<=40 else
    #                    '폭우' if x<=50 else
    #                    '집중호우' if x<=70 else
    #                    '자연재해급' for x in d['강수량(mm)']]
    
    # (7) 체감온도, 불쾌지수
    d['wind_chill'] = 13.12 + 0.6215*d['기온(C)'] - 11.37*(d['풍속(m/s)']**0.16) + 0.3965*d['기온(C)']*(d['풍속(m/s)']**0.16)
    d['discomfort_index'] = (9/5)*d['기온(C)'] - 0.55*(1-d['습도(%)']/100)*((9/5)*d['기온(C)']-26) + 32
    d['weather_condition'] = ['Hot and Dry'    if (temperature>=30) and (precipitation<1.0) else
                              'Hot and Rainy'  if (temperature>=30) and (precipitation>=1.0) else
                              'Warm and Dry'   if (25<=temperature<30) and (precipitation<1.0) else
                              'Warm and Rainy' if (25<=temperature<30) and (precipitation>=1.0) else
                              'Cool and Dry'   if (temperature<25) and (precipitation<1.0) else
                              'Cool and Rainy' for temperature,precipitation in d[['기온(C)','강수량(mm)']].values]
    
    # 컬럼재배열
    key_cols = ['건물번호','year','month','day','hour','weekday','weekend']
    target_col = '전력소비량(kWh)'
    other_cols = [col for col in d.columns if col not in key_cols+[target_col]]
    if target_col in d.columns:
        d = d[key_cols+other_cols+[target_col]]
    else:
        d = d[key_cols+other_cols]

    return d

In [15]:
# Null이 많은 컬럼들 제거
info_df.drop(columns=['태양광용량(kW)','ESS저장용량(kWh)','PCS용량(kW)'],inplace=True)

# 건물번호: int to string
train_df['건물번호'] = train_df['건물번호'].astype(str)
test_df ['건물번호'] = test_df ['건물번호'].astype(str)
info_df ['건물번호'] = info_df ['건물번호'].astype(str)

# 파생변수 생성
train_df = derived_features(train_df,info_df)
test_df  = derived_features(test_df ,info_df)

# train에만 있는 컬럼 삭제
train_df.drop(columns=['일조(hr)','일사(MJ/m2)'],inplace=True)

In [16]:
type_controller = TypeController(
    target_feature=CFG.TARGET,
    cat_features=['hour'],
    unuse_features=['year','month','day'],
    segment_feature=CFG.SEGMENT,
)
type_controller.fit(train_df)
global_assignment(type_controller.get_feature_type())

train_df = type_controller.transform(train_df)
test_df  = type_controller.transform(test_df)

In [17]:
train_df.head()

,건물번호,year,month,day,hour,weekday,weekend,기온(C),강수량(mm),풍속(m/s),습도(%),건물유형,연면적(m2),냉방면적(m2),is_holiday,sin_hour,cos_hour,time_of_day,is_rain,wind_chill,discomfort_index,weather_condition,전력소비량(kWh)
0,1.0,2022,6,1,0,2.0,0,18.6,0.0,0.9,42.0,건물기타,110634.0,39570.0,0.0,0.258819,0.965926,night,1,20.751584,63.09388,Cool and Dry,1085.28
1,1.0,2022,6,1,1,2.0,0,18.0,0.0,1.1,45.0,건물기타,110634.0,39570.0,0.0,0.500000,0.866025,night,1,20.008954,62.46400,Cool and Dry,1047.36
2,1.0,2022,6,1,2,2.0,0,17.7,0.0,1.5,45.0,건물기타,110634.0,39570.0,0.0,0.707107,0.707107,night,1,19.476910,62.08735,Cool and Dry,974.88
3,1.0,2022,6,1,3,2.0,0,16.7,0.0,1.4,48.0,건물기타,110634.0,39570.0,0.0,0.866025,0.500000,night,1,18.487958,60.89884,Cool and Dry,953.76
4,1.0,2022,6,1,4,2.0,0,18.4,0.0,2.8,43.0,건물기타,110634.0,39570.0,0.0,0.965926,0.258819,night,1,19.751534,62.88788,Cool and Dry,986.40


In [18]:
test_df.head()

,건물번호,year,month,day,hour,weekday,weekend,기온(C),강수량(mm),풍속(m/s),습도(%),건물유형,연면적(m2),냉방면적(m2),is_holiday,sin_hour,cos_hour,time_of_day,is_rain,wind_chill,discomfort_index,weather_condition
0,1.0,2022,8,25,0,3.0,0,23.5,0.0,2.2,72.0,건물기타,110634.0,39570.0,0.0,0.258819,0.965926,night,1,25.397063,71.78980,Cool and Dry
1,1.0,2022,8,25,1,3.0,0,23.0,0.0,0.9,72.0,건물기타,110634.0,39570.0,0.0,0.500000,0.866025,night,1,25.201620,71.02840,Cool and Dry
2,1.0,2022,8,25,2,3.0,0,22.7,0.0,1.5,75.0,건물기타,110634.0,39570.0,0.0,0.707107,0.707107,night,1,24.699788,70.81675,Cool and Dry
3,1.0,2022,8,25,3,3.0,0,22.1,0.0,1.3,78.0,건물기타,110634.0,39570.0,0.0,0.866025,0.500000,night,1,24.136018,70.11262,Cool and Dry
4,1.0,2022,8,25,4,3.0,0,21.8,0.0,1.0,77.0,건물기타,110634.0,39570.0,0.0,0.965926,0.258819,night,1,23.942400,69.56514,Cool and Dry


<br>

## Quantile values of Target

In [19]:
# feature engineering
calculator = CategoricalQuantileCalculator()
calculator.fit(
    data=train_df,
    test_data=test_df,
    target_feature=target_feature,
    cat_features=cat_features,
    subset_depth=CFG.SUBSET_DEPTH,
)
train_df2 = calculator.transform(train_df)
test_df2  = calculator.transform(test_df)

> Get quantiles of target by categorical features (depth=3)


Subset: hour + time_of_day + weather_condition: 100%|██████████| 7/7 [00:00<00:00, 10.39it/s]


In [21]:
type_controller = TypeController(
    target_feature=CFG.TARGET,
    cat_features=['hour'],
    unuse_features=['year','month','day'],
    segment_feature=CFG.SEGMENT,
)
type_controller.fit(train_df2)
global_assignment(type_controller.get_feature_type())

train_df2 = type_controller.transform(train_df2)
test_df2  = type_controller.transform(test_df2)

In [22]:
print(train_df.shape,'->',train_df2.shape)
train_df2.head()

(204000, 23) -> (204000, 51)


,건물번호,year,month,day,hour,weekday,weekend,기온(C),강수량(mm),풍속(m/s),습도(%),건물유형,연면적(m2),냉방면적(m2),is_holiday,sin_hour,cos_hour,time_of_day,is_rain,wind_chill,discomfort_index,weather_condition,전력소비량(kWh),hour_Avg,hour_Q25,hour_Q50,hour_Q75,time_of_day_Avg,time_of_day_Q25,time_of_day_Q50,time_of_day_Q75,weather_condition_Avg,weather_condition_Q25,weather_condition_Q50,weather_condition_Q75,hour&time_of_day_Avg,hour&time_of_day_Q25,hour&time_of_day_Q50,hour&time_of_day_Q75,hour&weather_condition_Avg,hour&weather_condition_Q25,hour&weather_condition_Q50,hour&weather_condition_Q75,time_of_day&weather_condition_Avg,time_of_day&weather_condition_Q25,time_of_day&weather_condition_Q50,time_of_day&weather_condition_Q75,hour&time_of_day&weather_condition_Avg,hour&time_of_day&weather_condition_Q25,hour&time_of_day&weather_condition_Q50,hour&time_of_day&weather_condition_Q75
0,1.0,2022,6,1,0,2.0,0,18.6,0.0,0.9,42.0,건물기타,110634.0,39570.0,0.0,0.258819,0.965926,night,1,20.751584,63.09388,Cool and Dry,1085.28,1832.909812,739.305,1185.54,2093.0400,1881.151324,807.45,1222.56,2160.48,2016.707077,905.76,1361.04,2460.96,1832.909812,739.305,1185.54,2093.0400,1709.219183,689.67,1104.84,1929.60,1732.251175,730.8,1124.64,1966.8,1709.219183,689.67,1104.84,1929.60
1,1.0,2022,6,1,1,2.0,0,18.0,0.0,1.1,45.0,건물기타,110634.0,39570.0,0.0,0.500000,0.866025,night,1,20.008954,62.46400,Cool and Dry,1047.36,1785.853802,704.160,1128.69,2063.0475,1881.151324,807.45,1222.56,2160.48,2016.707077,905.76,1361.04,2460.96,1785.853802,704.160,1128.69,2063.0475,1689.476585,660.96,1058.76,1927.38,1732.251175,730.8,1124.64,1966.8,1689.476585,660.96,1058.76,1927.38
2,1.0,2022,6,1,2,2.0,0,17.7,0.0,1.5,45.0,건물기타,110634.0,39570.0,0.0,0.707107,0.707107,night,1,19.476910,62.08735,Cool and Dry,974.88,1754.573766,691.110,1092.00,2031.4650,1881.151324,807.45,1222.56,2160.48,2016.707077,905.76,1361.04,2460.96,1754.573766,691.110,1092.00,2031.4650,1665.359986,645.12,1028.82,1890.30,1732.251175,730.8,1124.64,1966.8,1665.359986,645.12,1028.82,1890.30
3,1.0,2022,6,1,3,2.0,0,16.7,0.0,1.4,48.0,건물기타,110634.0,39570.0,0.0,0.866025,0.500000,night,1,18.487958,60.89884,Cool and Dry,953.76,1735.193839,678.105,1076.85,2008.6500,1881.151324,807.45,1222.56,2160.48,2016.707077,905.76,1361.04,2460.96,1735.193839,678.105,1076.85,2008.6500,1650.025511,643.05,1030.56,1841.94,1732.251175,730.8,1124.64,1966.8,1650.025511,643.05,1030.56,1841.94
4,1.0,2022,6,1,4,2.0,0,18.4,0.0,2.8,43.0,건물기타,110634.0,39570.0,0.0,0.965926,0.258819,night,1,19.751534,62.88788,Cool and Dry,986.40,1737.142635,687.420,1071.00,1987.2000,1881.151324,807.45,1222.56,2160.48,2016.707077,905.76,1361.04,2460.96,1737.142635,687.420,1071.00,1987.2000,1662.196602,655.56,1028.16,1865.07,1732.251175,730.8,1124.64,1966.8,1662.196602,655.56,1028.16,1865.07


<br>

## Target Transformation

In [23]:
def make_quantile(q):
    return train_df2.groupby(segment_feature)[target_feature].quantile([q]).reset_index().drop('level_1',axis=1).rename(columns={CFG.TARGET:f'q{str(int(100*q))}'})

d = make_quantile(0)\
    .merge(make_quantile(0.25),how='left',on=segment_feature)\
    .merge(make_quantile(0.50),how='left',on=segment_feature)\
    .merge(make_quantile(0.75),how='left',on=segment_feature)\
    .merge(make_quantile(1.00),how='left',on=segment_feature)

# d['건물번호'] = d['건물번호'].astype(float)
d.sort_values(segment_feature).tail(10)

,건물유형,q0,q25,q50,q75,q100
2,대학교,143.70,1212.480,2691.54,8088.09,25488.40
3,데이터센터,2046.60,2849.760,3371.40,8839.44,10272.24
4,백화점및아울렛,267.90,606.945,1946.10,3555.00,6439.80
5,병원,1056.90,2060.190,2693.10,3407.34,7808.16
6,상용,371.88,998.610,2392.20,3211.74,4943.52
7,아파트,206.82,613.080,1056.87,1675.26,6038.28
8,연구소,324.12,1327.650,2476.80,3696.00,7193.52
9,지식산업센터,415.44,1053.360,1583.34,2456.64,6788.88
10,할인마트,343.44,598.500,1556.16,1966.59,3171.12
11,호텔및리조트,0.00,923.220,1273.14,1717.44,4379.76


In [24]:
def ffill_outlier(dataset,segment_list):
    data = dataset.copy()
    data_list = []
    for segment in segment_list:
        d = data[data[segment_feature]==segment]
        d[target_feature][d[target_feature]<10] = np.nan
        n_ffill = d[target_feature].isnull().sum()
        if n_ffill!=0:
            print('segment={}: ffill {} values'.format(segment,n_ffill))
        d[target_feature].fillna(method='ffill',inplace=True)
        data_list.append(d)
    return pd.concat(data_list,axis=0).sort_index()

train_df3 = ffill_outlier(train_df2,train_df2[CFG.SEGMENT].unique())
test_df3  = test_df2.copy()

segment=호텔및리조트: ffill 2 values


In [25]:
# tmp = train_df3['전력소비량(kWh)']

# fig = plt.figure(figsize=(20,7))

# fig.add_subplot(121)
# sns.histplot(x=tmp,bins=100,kde=True)
# plt.grid()
# plt.title('(1) No Log')

# fig.add_subplot(122)
# sns.histplot(x=np.log(tmp),bins=100,kde=True)
# plt.grid()
# plt.title('(2) Log')

# plt.tight_layout()
# plt.show()

In [26]:
# segment_list = train_df[segment_feature].unique()
# i=0
# for segment in segment_list:
#     i+=1
#     d = train_df3[train_df3[segment_feature]==segment]
#     tmp = d['전력소비량(kWh)']
#     print('[{}/{}] {}'.format(i,len(segment_list),segment))
    
#     fig = plt.figure(figsize=(20,7))
    
#     fig.add_subplot(121)
#     sns.histplot(x=tmp,bins=100,kde=True)
#     plt.grid()
#     plt.title('(1) No Log')

#     fig.add_subplot(122)
#     sns.histplot(x=np.log(tmp),bins=100,kde=True)
#     plt.grid()
#     plt.title('(2) Log')
    
#     plt.tight_layout()
#     plt.show()

In [27]:
target_transform = TargetTransform(func=CFG.TARGET_TRANSFORMATION)
train_df3['전력소비량(kWh)'] = target_transform.fit_transform(
    target=train_df3['전력소비량(kWh)'],
    verbose=False,
)

<br>

## Imputation

In [28]:
null_info = train_df3\
    .isnull().sum()\
    .reset_index()\
    .rename(columns={'index':'feature',0:'null_cnt'})

null_info = null_info[null_info['null_cnt']!=0]
null_info['null_percentage'] = 100 * null_info['null_cnt'] / len(train_df3)

null_info.sort_values('null_cnt',ascending=False).round(2)

,feature,null_cnt,null_percentage
9,풍속(m/s),19,0.01
19,wind_chill,19,0.01
10,습도(%),9,0.00
20,discomfort_index,9,0.00


In [29]:
from sklearn.impute import SimpleImputer

impute_columns = ['풍속(m/s)','습도(%)']
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
train_df3[impute_columns] = imputer.fit_transform(train_df3[impute_columns])
test_df3 [impute_columns] = imputer.transform(test_df3[impute_columns])

In [30]:
def fix_derived_features(data):
    d = data.copy()
    d['wind_chill'] = 13.12 + 0.6215*d['기온(C)'] - 11.37*(d['풍속(m/s)']**0.16) + 0.3965*d['기온(C)']*(d['풍속(m/s)']**0.16)
    d['discomfort_index'] = (9/5)*d['기온(C)'] - 0.55*(1-d['습도(%)']/100)*((9/5)*d['기온(C)']-26) + 32
    return d

In [31]:
train_df3 = fix_derived_features(train_df3)
test_df3  = fix_derived_features(test_df3)

<br>

## Group Scaler

In [41]:
from sklearn.preprocessing import MinMaxScaler
scaler = GroupScaler(scaler=MinMaxScaler())
scaler.fit(
    data=train_df3,
    segment_feature=segment_feature,
    num_features=num_features,
)
train_df4 = scaler.transform(train_df3)
test_df4  = scaler.transform(test_df3)

[Transform] Segment: 호텔및리조트(12/12): 100%|██████████| 12/12 [00:01<00:00,  7.89it/s]    


<br>

## Memory Reduce

In [42]:
train_df5, _ = reduce_mem_usage(train_df4,verbose=True)
test_df5 , _ = reduce_mem_usage(test_df4 ,verbose=False)

[01/47] 건물번호: float64 -> float32
[02/47] year: int32 -> uint16
[03/47] month: int32 -> uint8
[04/47] day: int32 -> uint8
[05/47] weekday: float64 -> float32
[06/47] weekend: int64 -> uint8
[07/47] 기온(C): float64 -> float32
[08/47] 강수량(mm): float64 -> float32
[09/47] 풍속(m/s): float64 -> float32
[10/47] 습도(%): float64 -> float32
[11/47] 연면적(m2): float64 -> float32
[12/47] 냉방면적(m2): float64 -> float32
[13/47] is_holiday: float64 -> uint8
[14/47] sin_hour: float64 -> float32
[15/47] cos_hour: float64 -> float32
[16/47] is_rain: int64 -> uint8
[17/47] wind_chill: float64 -> float32
[18/47] discomfort_index: float64 -> float32
[19/47] 전력소비량(kWh): float64 -> float32
[20/47] hour_Avg: float64 -> float32
[21/47] hour_Q25: float64 -> float32
[22/47] hour_Q50: float64 -> float32
[23/47] hour_Q75: float64 -> float32
[24/47] time_of_day_Avg: float64 -> float32
[25/47] time_of_day_Q25: float64 -> float32
[26/47] time_of_day_Q50: float64 -> float32
[27/47] time_of_day_Q75: float64 -> float32
[28/47] 

In [43]:
train_df5.to_parquet('./out/train_df5.parquet')
test_df5 .to_parquet('./out/test_df5.parquet')

<br></br>

# Modeling

## Weighted Ensemble

In [44]:
train_df5 = pd.read_parquet('./out/train_df5.parquet')
test_df5 = pd.read_parquet('./out/test_df5.parquet')

In [45]:
train_fn = train_df5.copy()
test_fn  = test_df5 .copy()

In [46]:
from copy import deepcopy
from sklearn.base import BaseEstimator, RegressorMixin
from catboost import CatBoostRegressor, Pool
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error
import warnings
import time
import pickle

class WeightedEnsembleRegressor(BaseEstimator, RegressorMixin):
    def __init__(self,hyperparameters,weight=['equal','balanced'],inverse_transform=None):
        super().__init__()
        
        assert weight in ['equal','balanced'], \
            "weight must be one of ['equal','balanced']"
        
        self.hyperparameters = hyperparameters
        self.weight = weight
        self.inverse_transform = inverse_transform
        self._get_regressors()
    
    def _get_regressors(self):
        max_depth = 9
        n_jobs = -1
        
        params_ridge = {
            'alphas' : [1e-5, 1e-3, 1e-1, 1.0, 10.0, 100.0],
            'cv' : RepeatedKFold(n_splits=self.hyperparameters['n_splits'], n_repeats=3, 
                                 random_state=self.hyperparameters['random_state']),
        }
        
        params_lasso = {
            'alphas' : [1e-5, 1e-3, 1e-1, 1.0, 10.0, 100.0],
            'cv' : RepeatedKFold(n_splits=self.hyperparameters['n_splits'], n_repeats=3,
                                 random_state=self.hyperparameters['random_state']),
            'n_jobs' : n_jobs,
            #'max_iter' : 30000,
            'tol' : 0.001,
        }
        
        params_elasticnet = {
            'l1_ratio' : np.arange(0.1, 1, 0.1),
            'alphas' : [1e-5, 1e-3, 1e-1, 1.0, 10.0, 100.0],
            'cv' : RepeatedKFold(n_splits=self.hyperparameters['n_splits'], n_repeats=3,
                                 random_state=self.hyperparameters['random_state']),
            'n_jobs' : n_jobs,
            #'max_iter' : 30000,
            'tol' : 0.001,
        }
        
        params_catboost1 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : 'RMSE', #'MAE'
            'grow_policy' : 'Lossguide', # 'SymmetricTree','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            'max_depth' : self.hyperparameters['max_depth'],
            #'l2_leaf_reg' : 1,
        }
        
        params_catboost2 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : 'RMSE', #'MAE'
            'grow_policy' : 'Lossguide', # 'SymmetricTree','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            #'max_depth' : self.hyperparameters['max_depth'],
            'l2_leaf_reg' : 3,
        }
        
        params_catboost3 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : 'RMSE', #'MAE'
            'grow_policy' : 'SymmetricTree', # 'Lossguide','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            #'max_depth' : self.hyperparameters['max_depth'],
            'l2_leaf_reg' : 1,
        }
        
        params_catboost4 = {
            'random_state' : self.hyperparameters['random_state'],
            'iterations' : self.hyperparameters['iterations'],
            'early_stopping_rounds' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'loss_function' : 'RMSE', #'MAE'
            'grow_policy' : 'Depthwise', # 'SymmetricTree','Depthwise'
            'use_best_model' : True,
            'allow_writing_files' : False,
            'verbose' : 0,
            'max_depth' : self.hyperparameters['max_depth'],
            'l2_leaf_reg' : 1,
        }
    
        params_xgboost = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['xgb_iterations'],
            'early_stopping_rounds' : self.hyperparameters['xgb_early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['xgb_learning_rate'],
            'objective' : 'reg:squarederror',#'reg:absoluteerror',
            'verbosity' : 0,
            'max_depth': self.hyperparameters['max_depth'],
            'n_jobs' : n_jobs,
        }
    
        params_lightgbm = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['iterations'],
            'early_stopping_round' : self.hyperparameters['early_stopping_rounds'],
            'learning_rate' : self.hyperparameters['learning_rate'],
            'objective' : 'regression',
            'metric' : 'mean_absolute_error',
            'verbosity' : -1,
            'max_depth': self.hyperparameters['max_depth'],
            'n_jobs' : n_jobs,
        }
        
        params_extratrees = {
            'random_state' : self.hyperparameters['random_state'],
            'n_estimators' : self.hyperparameters['extratrees_iterations'],
            'criterion' : 'absolute_error',
            'verbose' : 0,
            'max_depth' : self.hyperparameters['max_depth'],
            'n_jobs' : n_jobs,
        }
        
        self.regressors = [
            #LinearRegression(),
            #RidgeCV(**params_ridge),
            #LassoCV(**params_lasso),
            #ElasticNetCV(**params_elasticnet),
            CatBoostRegressor(**params_catboost1),
            CatBoostRegressor(**params_catboost2),
            CatBoostRegressor(**params_catboost3),
            CatBoostRegressor(**params_catboost4),
            #XGBRegressor(**params_xgboost),
            #LGBMRegressor(**params_lightgbm),
            #ExtraTreesRegressor(**params_extratrees),
        ]
        self.regressors_name = [
            #'OLS',
            #'Ridge',
            #'Lasso',
            #'ElasticNet',
            'CatBoost1',
            'CatBoost2',
            'CatBoost3',
            'CatBoost4',
            #'XGBoost',
            #'LightGBM',
            #'ExtraTrees',
        ]
        
    def _set_zero_to_minimum(self,pred,minimum_value):
        pred = np.array(pred).flatten()
        if np.where(pred<0,1,0).sum()>0:
            pred = [x if x>0 else minimum_value for x in pred]
        pred = np.array(pred).flatten()
        return pred
    
    def _set_inf_to_maximum(self,pred,maximum_value):
        pred = np.array(pred).flatten()
        if np.where(pred==np.inf,1,0).sum()>0:
            pred = [x if x!=np.inf else maximum_value for x in pred]
        pred = np.array(pred).flatten()
        return pred
    
    def fit(self,X,y,eval_set,oh_set,cat_features,verbose=1):
        assert len(eval_set)==1, \
            "eval_set length must be 1. len(eval_set)={}".format(len(eval_set))
        assert len(oh_set)==1, \
            "oh_set length must be 1. len(oh_set)={}".format(len(oh_set))
        X_val, y_val = eval_set[0]
        X_oh, X_val_oh = oh_set[0]
        
        self.features = X.columns
        self.oh_features = X_oh.columns
        
        tr_true = np.array(y)    .flatten()
        va_true = np.array(y_val).flatten()
        if self.inverse_transform is not None:
            tr_true = self.inverse_transform(tr_true)
            va_true = self.inverse_transform(va_true)
        
        self.minimum_value = min(np.nanmin(y),np.nanmin(y_val))
        self.maximum_value = max(np.nanmax(y),np.nanmax(y_val))
        
        self.cat_features = cat_features
        self.scores = []
        self.weights = []
        self.fitting_elapsed = []
        if verbose:
            pbar = tqdm(zip(self.regressors_name,self.regressors),total=len(self.regressors))
        else:
            pbar = zip(self.regressors_name,self.regressors)
            
        fit_iter = 0
        for regressor_name,regressor in pbar:
            fit_iter+=1
            s = time.time()
            
            if verbose:
                pbar.set_description(name)
                
            if regressor_name in ['OLS','Ridge','Lasso','ElasticNet']:
                warnings.filterwarnings("ignore", category=UserWarning)
                regressor.fit(X_oh,y)
                tr_pred = regressor.predict(X_oh)
                va_pred = regressor.predict(X_val_oh)
            elif regressor_name.find('CatBoost')>=0:
                train_dataset = Pool(X,y,cat_features=cat_features)
                val_dataset   = Pool(X_val,y_val,cat_features=cat_features)
                regressor.fit(
                    train_dataset,
                    eval_set=val_dataset,
                    #metric_period=CFG.EPOCHS//5,
                )
                tr_pred = regressor.predict(train_dataset)
                va_pred = regressor.predict(val_dataset)
            elif regressor_name=='XGBoost':
                regressor.fit(
                    X_oh,y,
                    eval_set=[(X_val_oh,y_val)],
                    verbose=0,
                )
                tr_pred = regressor.predict(X_oh)
                va_pred = regressor.predict(X_val_oh)
            elif regressor_name=='LightGBM':
                warnings.filterwarnings("ignore", category=UserWarning)
                X_tmp = X.copy()
                X_val_tmp = X_val.copy()
                for col in cat_features:
                    X_tmp[col]     = X_tmp[col]    .astype('category')
                    X_val_tmp[col] = X_val_tmp[col].astype('category')
                regressor.fit(
                    X_tmp,y,
                    eval_set=[(X_val_tmp,y_val)],
                    verbose=-1,
                )
                tr_pred = regressor.predict(X_tmp)
                va_pred = regressor.predict(X_val_tmp)
            elif regressor_name=='ExtraTrees':
                regressor.fit(X_oh,y)
                tr_pred = regressor.predict(X_oh)
                va_pred = regressor.predict(X_val_oh)
            else:
                raise ValueError('Unknown Regressor: {}'.format(regressor_name))
            
            tr_pred = np.array(tr_pred).flatten()
            va_pred = np.array(va_pred).flatten()
            
            tr_pred = self._set_zero_to_minimum(tr_pred,self.minimum_value)
            va_pred = self._set_zero_to_minimum(va_pred,self.minimum_value)
            
            if self.inverse_transform is not None:
                tr_pred = self.inverse_transform(tr_pred)
                va_pred = self.inverse_transform(va_pred)
                tr_pred = self._set_inf_to_maximum(tr_pred,self.maximum_value)
                va_pred = self._set_inf_to_maximum(va_pred,self.maximum_value)
            
            tr_score = smape(pred=tr_pred,true=tr_true)
            va_score = smape(pred=va_pred,true=va_true)
            e = time.time()
            self.scores.append(va_score)
            self.weights.append(1/va_score)
            self.fitting_elapsed.append(e-s)
            
            blank = ' '*(11-len(regressor_name))
            fit_progress = '[{}/{}] {}{}: loss={:.3f}, val_loss={:.3f}, elasped={:.1f}s'\
                .format(fit_iter,len(self.regressors),regressor_name,blank,tr_score,va_score,e-s)
            print(fit_progress)
        
        if self.weight=='equal':
            self.weights = np.array([1.0 for _ in self.regressors])
        self.weights /= sum(self.weights)
        
        tr_pred = self.predict(X,X_oh)
        va_pred = self.predict(X_val,X_val_oh)
        
        ## -> self.predict에서 inverse_transform 해줌
        # if self.inverse_transform is not None:
        #     tr_pred = self.inverse_transform(tr_pred)
        #     va_pred = self.inverse_transform(va_pred)
        
        ens_tr_score = smape(true=tr_true,pred=tr_pred)
        ens_va_score = smape(true=va_true,pred=va_pred)
        
        ens_fit_progress = "<Weighted Ensemble(weight='{}')> loss={:.3f}, val_loss={:.3f}, elasped={:.1f}s"\
            .format(self.weight,ens_tr_score,ens_va_score,sum(self.fitting_elapsed))
        print(ens_fit_progress)
        
        self.ensemble_score = ens_va_score
        
    def predict(self,X,X_oh):
        assert len(X)==len(X_oh), \
            "X and X_oh must be same length"
        
        pred_list = []
        for regressor_name,regressor in zip(self.regressors_name,self.regressors):
            if regressor_name in ['OLS','Ridge','Lasso','ElasticNet','XGBoost','ExtraTrees']:
                dataset = X_oh.copy()
            elif regressor_name.find('CatBoost')>=0:
                dataset = Pool(X,cat_features=self.cat_features)
            elif regressor_name=='LightGBM':
                dataset = X.copy()
                for col in self.cat_features:
                    dataset[col] = dataset[col].astype('category')
            else:
                raise ValueError('Unknown Regressor: {}'.format(regressor_name))
            
            y_pred = regressor.predict(dataset)
            y_pred = self._set_zero_to_minimum(y_pred,self.minimum_value)
            
            pred_list.append(y_pred)
            
        final_pred = np.zeros(len(X))
        for pred,weight in zip(pred_list,self.weights):
            final_pred += np.array(pred)*weight
            
        if self.inverse_transform is not None:
            final_pred = self.inverse_transform(np.array(final_pred))
            final_pred = self._set_inf_to_maximum(final_pred,self.maximum_value)
            
        return final_pred
    
    def save_model(self,path):
        save_dict = {
            'cat_features' : self.cat_features,
            'weights' : self.weights,
            'fitting_elapsed' : self.fitting_elapsed,
            'regressors' : self.regressors,
            'minimum_value' : self.minimum_value,
            'maximum_value' : self.maximum_value,
            'features' : self.features,
            'oh_features' : self.oh_features,
            'scores' : self.scores,
            'hyperparameters' : self.hyperparameters,
            'inverse_transform' : self.inverse_transform,
            'ensemble_score' : self.ensemble_score,
        }
        with open(path, 'wb') as f:
            pickle.dump(save_dict, f, protocol=pickle.HIGHEST_PROTOCOL)
            
    def load_model(self,path):
        with open(path, 'rb') as f:
            save_dict = pickle.load(f)
            self.cat_features = save_dict['cat_features']
            self.weights = save_dict['weights']
            self.fitting_elapsed = save_dict['fitting_elapsed']
            self.regressors = save_dict['regressors']
            self.minimum_value = save_dict['minimum_value']
            self.maximum_value = save_dict['maximum_value']
            self.features = save_dict['features']
            self.oh_features = save_dict['oh_features']
            self.scores = save_dict['scores']
            self.hyperparameters = save_dict['hyperparameters']
            self.inverse_transform = save_dict['inverse_transform']
            self.ensemble_score = save_dict['ensemble_score']

In [47]:
from sklearn.model_selection import KFold

In [48]:
def split_segment(data,segment,seg):
    return data[data[segment]==seg]

In [49]:
hyperparameters = {
    'n_splits' : 5,
    'max_depth' : 9,
    'random_state' : 0,
    'learning_rate' : 0.05,
    'iterations' : 30000,
    'early_stopping_rounds' : 300,
    'xgb_learning_rate' : 0.3,         # default=0.3
    'xgb_iterations' : 3000,           # default=100
    'xgb_early_stopping_rounds' : 30,
    'extratrees_iterations' : 100,     # default=100
}

In [50]:
X      = train_fn.drop(target_feature,axis=1)
X_test = test_fn.copy()
y      = train_fn[target_feature]

ohe = OneHotEncoder()
ohe.fit(X,cat_features)
X_oh      = ohe.transform(X)
X_test_oh = ohe.transform(X_test)

segment_list = X[segment_feature].unique()

In [ ]:
%%time
# 2시간

save_path = './mc/kf_weiens_catall_seg건물유형/segment{}_k{}.pickle'

models = {}
feature_info = {}
scores = []

s_i = 0
for segment in tqdm(segment_list):
    s_i+=1
    gc_collect_all(verbose=False)
    
    # segment에 해당하는 데이터추출
    _X         = split_segment(X        ,segment_feature,segment)
    _X_oh      = split_segment(X_oh     ,segment_feature,segment)
    _X_test    = split_segment(X_test   ,segment_feature,segment)
    _X_test_oh = split_segment(X_test_oh,segment_feature,segment)
    _y         = y[X[segment_feature]==segment]
    
    # kfold
    kf = KFold(n_splits=CFG.N_SPLITS,random_state=1000*s_i+CFG.SEED,shuffle=True)
    
    # unique인 컬럼 제외
    # (1) X
    unique_info = _X.nunique()
    unique_cols = unique_info[unique_info==1].index.tolist()
    if len(unique_cols)>0:
        _X = _X.drop(unique_cols,axis=1)
    # (2) X_oh
    unique_info = _X_oh.nunique()
    unique_cols = unique_info[unique_info==1].index.tolist()
    if len(unique_cols)>0:
        _X_oh = _X_oh.drop(unique_cols,axis=1)
        
    # categorical feature에서 unique인 컬럼을 제외
    fixed_cat_features = [col for col in cat_features if col in _X.columns]
    
    # progress
    progress = '> [{}/{}] Segment: {}, Length: {}'.format(s_i,len(segment_list),segment,len(_X))
    print('-'*80)
    print(progress)
    print('-'*80)
    
    _models = []
    _scores = []
    k=0
    for tr_idx,val_idx in kf.split(_X,_y):
        k+=1
        print('> [K-Fold] {}/{}'.format(k,CFG.N_SPLITS))
        
        # kfold dataset
        X_tr   , X_va    = _X   .iloc[tr_idx], _X   .iloc[val_idx]
        X_tr_oh, X_va_oh = _X_oh.iloc[tr_idx], _X_oh.iloc[val_idx]
        y_tr   , y_va    = _y   .iloc[tr_idx], _y   .iloc[val_idx]

        # define the model
        ensemble_model = WeightedEnsembleRegressor(
            hyperparameters=hyperparameters,
            weight='balanced',
            inverse_transform=target_transform.inverse_transform,
        )

        # fit the model
        ensemble_model.fit(
            X_tr,y_tr,
            eval_set=[(X_va,y_va)],
            oh_set=[(X_tr_oh,X_va_oh)],
            cat_features=fixed_cat_features,
            verbose=0,
        )

        # save the model
        ensemble_model.save_model(save_path.format(segment,k))

        # prediction
        y_pred = ensemble_model.predict(X_va,X_va_oh).flatten()
        y_true = target_transform.inverse_transform(y_va.values)
        
        # caculate score
        score = mean_absolute_error(y_true=y_true,y_pred=y_pred)
        
        # append inner loop
        _models.append(ensemble_model)
        _scores.append([segment,k,len(X_tr),len(X_va),score])
    
    # kfold result -> plotting
    te_preds = []
    for m in _models:
        te_pred = m.predict(_X_test[X_tr.columns],_X_test_oh[X_tr_oh.columns]).flatten()
        te_preds.append(te_pred)

    te_pred = np.mean(te_preds,axis=0)

    y1 = target_transform.inverse_transform(_y).tolist()
    y2 = te_pred.tolist()

    x1 = range(len(y1))
    x2 = [i+max(x1) for i in range(len(y2))]

    plt.figure(figsize=(15,7))
    sns.lineplot(x=x1,y=y1,color='gray')
    sns.lineplot(x=x2,y=y2,color='red')
    plt.grid()
    plt.show()
        
    # append outer loop
    models[segment] = _models
    scores.append(_scores)
    feature_info[segment] = {
        'cat_features':fixed_cat_features,
        'features':_X.columns.tolist(),
        'oh_features':_X_oh.columns.tolist(),
    }

  0%|          | 0/12 [00:00<?, ?it/s]

--------------------------------------------------------------------------------
> [1/12] Segment: 건물기타, Length: 30600
--------------------------------------------------------------------------------
> [K-Fold] 1/5


In [ ]:
score_df = pd.concat([pd.DataFrame(s,columns=['segment','k','n_train','n_val','mae']) for s in scores],axis=0)
score_df

In [ ]:
def load_model(hyperparameters,path):
    ensemble_model = WeightedEnsembleRegressor(
        hyperparameters=hyperparameters,
        weight='balanced',
        inverse_transform=target_transform.inverse_transform,
    )
    ensemble_model.load_model(path)
    return ensemble_model

In [ ]:
X      = train_fn.drop(target_feature,axis=1)
X_test = test_fn.copy()
y      = train_fn[target_feature]

ohe = OneHotEncoder()
ohe.fit(X,cat_features)
X_oh      = ohe.transform(X)
X_test_oh = ohe.transform(X_test)

segment_list = X[segment_feature].unique()

In [ ]:
tr_pred_list = []
te_pred_list = []
for segment in tqdm(segment_list):
    gc_collect_all(verbose=False)
    
    # segment에 해당하는 데이터추출
    _X         = split_segment(X        ,segment_feature,segment)
    _X_oh      = split_segment(X_oh     ,segment_feature,segment)
    _X_test    = split_segment(X_test   ,segment_feature,segment)
    _X_test_oh = split_segment(X_test_oh,segment_feature,segment)
    
    path = './mc/kf_weiens_catall_seg건물유형/segment{}_k{}.pickle'
    model_list = [load_model(hyperparameters,path.format(segment,k)) for k in range(1,6)]
    features = model_list[0].features
    oh_features = model_list[0].oh_features
    _X = _X[features]
    _X_oh = _X_oh[oh_features]
    _X_test = _X_test[features]
    _X_test_oh = _X_test_oh[oh_features]

    tr_pred = np.mean([model.predict(_X,_X_oh) for model in model_list],axis=0)
    tr_pred = pd.DataFrame(tr_pred,index=_X.index)
    tr_pred_list.append(tr_pred)
    
    te_pred = np.mean([model.predict(_X_test[_X.columns],_X_test_oh[_X_oh.columns]) for model in model_list],axis=0)
    te_pred = pd.DataFrame(te_pred,index=_X_test.index)
    te_pred_list.append(te_pred)

In [ ]:
tr_pred_df = pd.concat(tr_pred_list,axis=0).sort_index()
te_pred_df = pd.concat(te_pred_list,axis=0).sort_index()

In [ ]:
smape(pred=tr_pred_df.values.flatten(),true=target_transform.inverse_transform(y))

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['answer'] = te_pred_df.values

In [ ]:
submit.head()

In [ ]:
# public :  (log)
submit.to_csv('./out/17_weiens_seg_catall_seg건물유형.csv',index=False)

<br>

## Stacking

In [ ]:
hyperparameters = {
    'n_splits' : 5,
    'max_depth' : 9,
    'random_state' : 0,
    'learning_rate' : 0.01,
    'iterations' : 30000,
    'early_stopping_rounds' : 300,
    'xgb_learning_rate' : 0.03,         # default=0.3
    'xgb_iterations' : 3000,           # default=100
    'xgb_early_stopping_rounds' : 30,
    'extratrees_iterations' : 100,     #default=100
}

In [ ]:
X      = train_fn.drop(target_feature,axis=1)
X_test = test_fn.copy()
y      = train_fn[target_feature]

ohe = OneHotEncoder()
ohe.fit(X,cat_features)
X_oh      = ohe.transform(X)
X_test_oh = ohe.transform(X_test)

segment_list = X[segment_feature].unique()

In [ ]:
%%time
# 2시간

save_path = './mc/kf_weiens_catall_seg건물유형/stacking_segment{}_k{}.pickle'

stacking_models = {}
stacking_feature_info = {}
stacking_scores = []

s_i = 0
for segment in tqdm(segment_list):
    s_i+=1
    gc_collect_all(verbose=False)
    
    # segment에 해당하는 데이터추출
    _X         = split_segment(X        ,segment_feature,segment)
    _X_oh      = split_segment(X_oh     ,segment_feature,segment)
    _X_test    = split_segment(X_test   ,segment_feature,segment)
    _X_test_oh = split_segment(X_test_oh,segment_feature,segment)
    _y         = y[X[segment_feature]==segment]
    
    path = './mc/kf_weiens_catall_seg건물유형/segment{}_k{}.pickle'
    model_list = [load_model(hyperparameters,path.format(segment,k)) for k in range(1,6)]
    
    features    = model_list[0].features
    oh_features = model_list[0].oh_features
    _X = _X[features]
    _X_oh = _X_oh[oh_features]
    _X_test = _X_test[features]
    _X_test_oh = _X_test_oh[oh_features]
    
    tr_pred = pd.DataFrame(index=_X.index)
    te_pred = pd.DataFrame(index=_X_test.index)
    for i,model in enumerate(model_list,1):
        tr_pred[f'pred{i}'] = model.predict(_X,_X_oh)
        te_pred[f'pred{i}'] = model.predict(_X_test,_X_test_oh)

    _X_new         = pd.concat([_X        ,tr_pred],axis=1)
    _X_oh_new      = pd.concat([_X_oh     ,tr_pred],axis=1)
    _X_test_new    = pd.concat([_X_test   ,te_pred],axis=1)
    _X_test_oh_new = pd.concat([_X_test_oh,te_pred],axis=1)
    
    # categorical feature에서 unique인 컬럼을 제외
    fixed_cat_features = [col for col in cat_features if col in _X_new.columns]
    
    # progress
    progress = '> [{}/{}] Segment: {}, Length: {}'.format(s_i,len(segment_list),segment,len(_X))
    print('-'*80)
    print(progress)
    print('-'*80)
    
    # kfold
    kf = KFold(n_splits=CFG.N_SPLITS,random_state=1000*s_i+CFG.SEED,shuffle=True)
    
    _models = []
    _scores = []
    k=0
    for tr_idx,val_idx in kf.split(_X,_y):
        k+=1
        print('> [K-Fold] {}/{}'.format(k,CFG.N_SPLITS))
        
        # kfold dataset
        X_tr   , X_va    = _X_new   .iloc[tr_idx], _X_new   .iloc[val_idx]
        X_tr_oh, X_va_oh = _X_oh_new.iloc[tr_idx], _X_oh_new.iloc[val_idx]
        y_tr   , y_va    = _y       .iloc[tr_idx], _y       .iloc[val_idx]

        # define the model
        ensemble_model = WeightedEnsembleRegressor(
            hyperparameters=hyperparameters,
            weight='balanced',
            inverse_transform=target_transform.inverse_transform,
        )

        # fit the model
        ensemble_model.fit(
            X_tr,y_tr,
            eval_set=[(X_va,y_va)],
            oh_set=[(X_tr_oh,X_va_oh)],
            cat_features=fixed_cat_features,
            verbose=0,
        )

        # save the model
        ensemble_model.save_model(save_path.format(segment,k))

        # prediction
        y_pred = ensemble_model.predict(X_va,X_va_oh).flatten()
        y_true = target_transform.inverse_transform(y_va.values)
        
        # caculate score
        score = mean_absolute_error(y_true=y_true,y_pred=y_pred)
        
        # append inner loop
        _models.append(ensemble_model)
        _scores.append([segment,k,len(X_tr),len(X_va),score])
    
    # kfold result -> plotting
    te_preds = []
    for m in _models:
        te_pred = m.predict(_X_test_new[X_tr.columns],_X_test_oh_new[X_tr_oh.columns]).flatten()
        te_preds.append(te_pred)

    te_pred = np.mean(te_preds,axis=0)

    y1 = target_transform.inverse_transform(_y).tolist()
    y2 = te_pred.tolist()

    x1 = range(len(y1))
    x2 = [i+max(x1) for i in range(len(y2))]

    plt.figure(figsize=(15,7))
    sns.lineplot(x=x1,y=y1,color='gray')
    sns.lineplot(x=x2,y=y2,color='red')
    plt.grid()
    plt.show()
        
    # append outer loop
    stacking_models[segment] = _models
    stacking_scores.append(_scores)
    stacking_feature_info[segment] = {
        'cat_features':fixed_cat_features,
        'features':_X_new.columns.tolist(),
        'oh_features':_X_oh_new.columns.tolist(),
    }

In [ ]:
score_df = pd.concat([
    pd.DataFrame(s,columns=['segment','k','n_train','n_val','mae']) for s in stacking_scores
],axis=0)
score_df.sort_values('mae',ascending=False).head()

In [ ]:
X      = train_fn.drop(target_feature,axis=1)
X_test = test_fn.copy()
y      = train_fn[target_feature]

ohe = OneHotEncoder()
ohe.fit(X,cat_features)
X_oh      = ohe.transform(X)
X_test_oh = ohe.transform(X_test)

segment_list = X[segment_feature].unique()

    # segment에 해당하는 데이터추출
    _X         = split_segment(X        ,segment_feature,segment)
    _X_oh      = split_segment(X_oh     ,segment_feature,segment)
    _X_test    = split_segment(X_test   ,segment_feature,segment)
    _X_test_oh = split_segment(X_test_oh,segment_feature,segment)
    _y         = y[X[segment_feature]==segment]
    
    path = './mc/kf_weiens_catall_seg건물유형/segment{}_k{}.pickle'
    model_list = [load_model(hyperparameters,path.format(segment,k)) for k in range(1,6)]
    
    features    = model_list[0].features
    oh_features = model_list[0].oh_features
    _X = _X[features]
    _X_oh = _X_oh[oh_features]
    _X_test = _X_test[features]
    _X_test_oh = _X_test_oh[oh_features]
    
    tr_pred = pd.DataFrame(index=_X.index)
    te_pred = pd.DataFrame(index=_X_test.index)
    for i,model in enumerate(model_list,1):
        tr_pred[f'pred{i}'] = model.predict(_X,_X_oh)
        te_pred[f'pred{i}'] = model.predict(_X_test,_X_test_oh)

In [ ]:
tr_pred_list = []
te_pred_list = []
for segment in tqdm(segment_list):
    gc_collect_all(verbose=False)
    
    # segment에 해당하는 데이터추출
    _X         = split_segment(X        ,segment_feature,segment)
    _X_oh      = split_segment(X_oh     ,segment_feature,segment)
    _X_test    = split_segment(X_test   ,segment_feature,segment)
    _X_test_oh = split_segment(X_test_oh,segment_feature,segment)
    
    path = './mc/kf_weiens_catall_seg건물유형/stacking_segment{}_k{}.pickle'
    model_list = [load_model(hyperparameters,path.format(segment,k)) for k in range(1,6)]
    
    org_path = './mc/kf_weiens_catall_seg건물유형/segment{}_k{}.pickle'
    org_model_list = [load_model(hyperparameters,org_path.format(segment,k)) for k in range(1,6)]
    org_features = org_model_list[0].features
    org_oh_features = org_model_list[0].oh_features
    for i,model in enumerate(org_model_list):
        tr_pred_ = model.predict(_X[org_features],_X_oh[org_oh_features])
        _X[f'pred{i+1}']    = tr_pred_
        _X_oh[f'pred{i+1}'] = tr_pred_
        te_pred_ = model.predict(_X_test[org_features],_X_test_oh[org_oh_features])
        _X_test[f'pred{i+1}']    = te_pred_
        _X_test_oh[f'pred{i+1}'] = te_pred_
    
    _X = _X[features]
    _X_oh = _X_oh[oh_features]
    _X_test = _X_test[features]
    _X_test_oh = _X_test_oh[oh_features]

    tr_pred = np.mean([model.predict(_X,_X_oh) for model in model_list],axis=0)
    tr_pred = pd.DataFrame(tr_pred,index=_X.index)
    tr_pred_list.append(tr_pred)
    
    te_pred = np.mean([model.predict(_X_test[_X.columns],_X_test_oh[_X_oh.columns]) for model in model_list],axis=0)
    te_pred = pd.DataFrame(te_pred,index=_X_test.index)
    te_pred_list.append(te_pred)

In [ ]:
tr_pred_df = pd.concat(tr_pred_list,axis=0).sort_index()
te_pred_df = pd.concat(te_pred_list,axis=0).sort_index()

In [ ]:
smape(pred=tr_pred_df.values.flatten(),true=target_transform.inverse_transform(y))

In [ ]:
submit = pd.read_csv('./data/sample_submission.csv')
submit['answer'] = te_pred_df.values

In [ ]:
submit.head()

In [ ]:
# public : 6.0993105553 (log)
submit.to_csv('./out/18_weiens_seg_stacking_catall_seg건물유형.csv',index=False)

In [ ]:
# import pandas as pd
# a = pd.read_csv('./out/12_weiens_seg_catall.csv')
# b = pd.read_csv('./out/13_weiens_seg_stacking_catall.csv')

# submit = pd.read_csv('./data/sample_submission.csv')
# submit['answer'] = 0.3*a['answer'] + 0.7*b['answer']

In [ ]:
# # public : mean     - 6.1023049459 (log)
# # public : 0.3, 0.7 - 6.0998997065 (log)
# submit.to_csv('./out/14_weiens_seg_catall_wei_mean(estimators,stacking).csv',index=False)